In [1]:
using_collab = True

if using_collab :
  from google.colab import drive
  drive.mount('/content/drive')

file_path = "/content/drive/MyDrive" #file_path를 설정해주세요.

Mounted at /content/drive


In [2]:
import pandas as pd
import tqdm
import numpy
import json

In [5]:
rawdata = pd.read_csv(file_path + '/KISTI/kisti_final_data/final_test_data.csv')

In [7]:
#다나와 최종 prompt이용해서 task 1 상품데이터 구성
rawdata = rawdata[rawdata['data'].isin(['danawa2-1,2'])]

In [8]:
len(rawdata)

1005

In [9]:
#예시확인
rawdata['input'][40945]

'이 상품의 이름은 닥터그루트 애딕트 피오니 루즈 샴푸 680ml (1개) 이고, 해당 제품의 상세 설명은 제조회사 : LG생활건강, 등록년월 : 2019년 06월, nan : nan, 약산성(pH4.5~6.5) : O, 설페이트프리 : O, 실리콘프리 : O, 기능성인증 : 탈모증상완화, 적합성평가인증 : 상세설명 / 판매 사이트 문의 인증번호 확인, 안전확인인증 : 상세설명 / 판매 사이트 문의 인증번호 확인'

In [10]:
rawdata['output'][40945]

'다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 패션/잡화/뷰티 level 2: 헤어케어 level 3: 샴푸 level 4: 탈모샴푸 카테고리로 분류할 수 있습니다.'

In [11]:
rawdata = rawdata[rawdata['category'].isin(['task2-1'])]

In [12]:
rawdata.columns

Index(['instruction', 'input', 'output', 'category', 'index', 'data',
       'product_key'],
      dtype='object')

### 예시  
**instruction**  


**input**  
이 상품의 이름은 닥터그루트 애딕트 피오니 루즈 샴푸 680ml (1개) 이고, 해당 제품의 상세 설명은
제조회사 : LG생활건강, 등록년월 : 2019년 06월, nan : nan, 약산성(pH4.5~6.5) : O, 설페이트프리 : O,
실리콘프리 : O, 기능성인증 : 탈모증상완화, 적합성평가인증 : 상세설명 / 판매 사이트 문의 인증번호 확인,
안전확인인증 : 상세설명 / 판매 사이트 문의 인증번호 확인
                
**output**  
다음 상품을 계층적으로 4단계로 분류하였을 때,
level 1: 패션/잡화/뷰티 level 2: 헤어케어 level 3: 샴푸 level 4: 탈모샴푸 카테고리로 분류할 수 있습니다.

In [14]:
#instruction #input #output

data = pd.DataFrame()

# Generate 'context' column
data['input'] = rawdata['input']

# Generate 'response' column
data['output'] =rawdata['output']
data['instruction'] = ""


data = data[['instruction','input','output']]

data.head()

,instruction,input,output
39941,,"이 상품의 이름은 닥터포헤어 폴리젠 바이오3 샴푸 750ml (1개) 이고, 해당 ...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 패션/잡화/뷰티 l..."
39942,,"이 상품의 이름은 청미정 흑곡 샴푸 500ml (1개) 이고, 해당 제품의 상세 설...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 패션/잡화/뷰티 l..."
39943,,"이 상품의 이름은 굿매너 KF80 소형 (100개) 이고, 해당 제품의 상세 설명은...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 생활/주방/건강 l..."
39944,,"이 상품의 이름은 헤드스파 두피마사지기 (1개) 이고, 해당 제품의 상세 설명은 제...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 가전/TV leve..."
39945,,"이 상품의 이름은 닥터방기원 쿨 샴푸 1000ml (1개) 이고, 해당 제품의 상세...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 패션/잡화/뷰티 l..."


In [15]:
data['input']

39941    이 상품의 이름은 닥터포헤어 폴리젠 바이오3 샴푸 750ml (1개) 이고, 해당 ...
39942    이 상품의 이름은 청미정 흑곡 샴푸 500ml (1개) 이고, 해당 제품의 상세 설...
39943    이 상품의 이름은 굿매너 KF80 소형 (100개) 이고, 해당 제품의 상세 설명은...
39944    이 상품의 이름은 헤드스파 두피마사지기 (1개) 이고, 해당 제품의 상세 설명은 제...
39945    이 상품의 이름은 닥터방기원 쿨 샴푸 1000ml (1개) 이고, 해당 제품의 상세...
                               ...                        
40941    이 상품의 이름은 경성오토비스 오토맙 A5-5000 (일반구매) 이고, 해당 제품의...
40942    이 상품의 이름은 리빙앤 워터스핀2 NBA-AS2200 이고, 해당 제품의 상세 설...
40943    이 상품의 이름은 For LG LGA-UV3500 이고, 해당 제품의 상세 설명은 ...
40944    이 상품의 이름은 대웅모닝컴 DWO-500VC (일반구매) 이고, 해당 제품의 상세...
40945    이 상품의 이름은 닥터그루트 애딕트 피오니 루즈 샴푸 680ml (1개) 이고, 해...
Name: input, Length: 1005, dtype: object

In [17]:
data.to_csv(file_path + '/KISTI/task12_generating/danawa_product.csv')

In [18]:
danawa_product_csv = pd.read_csv(file_path + '/KISTI/task12_generating/danawa_product.csv')

In [19]:
danawa_product_csv

,Unnamed: 0,instruction,input,output
0,39941,NaN,"이 상품의 이름은 닥터포헤어 폴리젠 바이오3 샴푸 750ml (1개) 이고, 해당 ...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 패션/잡화/뷰티 l..."
1,39942,NaN,"이 상품의 이름은 청미정 흑곡 샴푸 500ml (1개) 이고, 해당 제품의 상세 설...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 패션/잡화/뷰티 l..."
2,39943,NaN,"이 상품의 이름은 굿매너 KF80 소형 (100개) 이고, 해당 제품의 상세 설명은...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 생활/주방/건강 l..."
3,39944,NaN,"이 상품의 이름은 헤드스파 두피마사지기 (1개) 이고, 해당 제품의 상세 설명은 제...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 가전/TV leve..."
4,39945,NaN,"이 상품의 이름은 닥터방기원 쿨 샴푸 1000ml (1개) 이고, 해당 제품의 상세...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 패션/잡화/뷰티 l..."
...,...,...,...,...
1000,40941,NaN,"이 상품의 이름은 경성오토비스 오토맙 A5-5000 (일반구매) 이고, 해당 제품의...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 가전/TV leve..."
1001,40942,NaN,"이 상품의 이름은 리빙앤 워터스핀2 NBA-AS2200 이고, 해당 제품의 상세 설...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 가전/TV leve..."
1002,40943,NaN,"이 상품의 이름은 For LG LGA-UV3500 이고, 해당 제품의 상세 설명은 ...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 가전/TV leve..."
1003,40944,NaN,"이 상품의 이름은 대웅모닝컴 DWO-500VC (일반구매) 이고, 해당 제품의 상세...","다음 상품을 계층적으로 4단계로 분류하였을 때, level 1: 가전/TV leve..."


In [20]:
# 데이터프레임을 딕셔너리로 변환하여 저장할 리스트 생성
data_list1 = []

# 데이터프레임의 각 행에 대해 딕셔너리 생성 후 리스트에 추가
for _, row in danawa_product_csv.iterrows():
    data_dict1 = {
        'instruction' : "",
        'input': row['input'] ,
        'output': row['output']
    }
    data_list1.append(data_dict1)

In [21]:
#딕셔너리 리스트를 JSON파일로 저장
with open(file_path + '/KISTI/task12_generating/danawa_product.json', 'w',encoding='utf-8') as f:
    json.dump(data_list1, f, ensure_ascii=False)